In [1]:
import site

site.getsitepackages()

['/home/maidari/miniconda3/envs/myenv311/lib/python3.11/site-packages']

In [2]:
from transformers.models.bert.modeling_bert import (
    BertConfig,
    BertPreTrainedModel,
    BertAttention,
    SequenceClassifierOutput,
    BertForSequenceClassification,
    BertModel,
    BertEncoder,
    BertEmbeddings,
    BertPooler,
    BertLayer
)

In [3]:
model_name = "google-bert/bert-base-uncased"

In [ ]:
class BertMoEModel(BertModel):
    def __init__(self, config, add_pooling_layer=True):
        super(BertModel, self).__init__(config)
        self.config = config
        self.embeddings = BertEmbeddings(config)
        self.encoder = BertMoEEncoder(config)  # Используем наш MoE-энкодер
        self.pooler = BertPooler(config) if add_pooling_layer else None
        self.init_weights()

In [4]:
class BertMoEForSequenceClassification(BertForSequenceClassification):
    def __init__(self, config):
        super(BertForSequenceClassification, self).__init__(config)
        self.num_labels = config.num_labels
        self.bert = BertMoEModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        
        # Пропускаем через модель
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        # Рассчитываем основной loss
        loss = None
        if labels is not None:
            if self.num_labels == 1:
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
        
        # Добавляем вспомогательный loss от MoE
        aux_loss = 0.0
        if self.training:
            for layer in self.bert.encoder.layer:
                aux_loss += layer.moe.aux_loss
            if loss is not None:
                loss += aux_loss

        # Формируем вывод
        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output
        
        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [ ]:
config = BertConfig.from_pretrained(model_name)
config.moe_num_experts = 4
config.moe_top_k = 1
config.moe_aux_loss_coef = 0.1

model = BertMoEForSequenceClassification(config)